##### Importando todas as bibliotecas nescessárias
<p> pandas: Pandas é uma biblioteca de manipulação e análise de dados. 

<p> re: Biblioteca para uso de expressões regulares.

<p> spellchecker: Utiliza um algoritmo de Distância de Levenshtein para encontrar permutações dentro de uma distância de edição de 2 da palavra original. Em seguida, compara todas as permutações (inserções, eliminações, substituições e transposições) com palavras conhecidas numa lista de frequência de palavras. As palavras que são encontradas com mais frequência na lista de frequência são mais provavelmente os resultados corretos.

<p> thefuzz: Oferece correspondência de cadeia de caracteres difusa para calcular as diferenças entre sequências. Licença: MIT. 

In [1]:
import pandas as pd
import re
from spellchecker import SpellChecker
from thefuzz import fuzz, process
from tqdm import tqdm 

# Definindo bibliotecas de chaves para sistemas
sistemas = {
    'Motor': {'motor', 'cilindro', 'biela', 'cárter', 'válvula', 'potência', 'cabeçote', 'turbina'},
    'Arrefecimento': {'radiador', 'água', 'líquido', 'arrefecimento', 'aditivo', 'temperatura'},
    'Cambio': {'câmbio', 'transmissão', 'embreagem', 'engrenagem', 'marcha'},
    'Elétrico': {'elétrico', 'bateria', 'alternador', 'motor de partida', 'lanterna', 'lâmpadas', 'iluminação', 'injeção', 'luz'},
    'Rodagem': {'rodagem', 'eixo', 'roda', 'pneu', 'borracha', 'rodizio'},
    'Cabine': {'cabine', 'interior', 'painel', 'bancos', 'parabrisa', 'assento'},
    'Pneumático': {'pneumático', 'vazamento de ar', 'mangueira'},
    'Chassi': {'chassi', 'estrutura', 'quadriciclo'},
    'Suspensão': {'suspensão', 'amortecedor', 'mola', 'bexiga', 'bexigao'},
    'Freio': {'freio', 'pastilha', 'disco', 'sistema de freio', 'cuica'},
    'Direção': {'direção', 'volante', 'coluna de direção'},
    'Diferencial': {'diferencial', 'eixo diferencial'},
    'Acessórios': {'retrovisor', 'palheta', 'ar condicionado', 'parabarro', 'paralama', 'retrovisor'},
    'Combustível': {'tanque', 'arla', 'diesel'},
    'Estrutura':{'malhal','fueiro', 'paramotociclista', 'regua', 'vigia', 'chapa'}
}

# Definindo palavras-chave para falha operacional
falhas_operacionais = {
    'Falha': {'fueiro', 'malhal', 'motoqueiro', 'tombamento', 'parabrisa', 'retrovisor'}
}

# Inicializar o corretor ortográfico
spell = SpellChecker(language='pt')

# Compilar expressão regular para remover pontuação uma vez
punctuation_regex = re.compile(r'[^\w\s]')

def corrigir_ortografia(texto):
    """
    Corrige a ortografia das palavras em um texto.
    
    Args:
        texto (str): Texto a ser corrigido.
    
    Returns:
        str: Texto com ortografia corrigida.
    """
    palavras = texto.split()
    palavras_corrigidas = []
    for palavra in palavras:
        # Remover pontuação
        palavra_clean = punctuation_regex.sub('', palavra)
        
        # Verificar se a palavra é alfabética e tem mais de 2 caracteres
        if palavra_clean.isalpha() and len(palavra_clean) > 2:
            # Verifica se a palavra está errada
            if palavra_clean not in spell:
                # Corrige a palavra
                palavra_corrigida = spell.correction(palavra_clean)
                # Caso a correção não seja confiável, mantém a palavra original
                if palavra_corrigida:
                    palavras_corrigidas.append(palavra_corrigida)
                else:
                    palavras_corrigidas.append(palavra_clean)
            else:
                palavras_corrigidas.append(palavra_clean)
        else:
            # Mantém a palavra original se não for alfabética ou for muito curta
            palavras_corrigidas.append(palavra_clean)
    return ' '.join(palavras_corrigidas)

def classificar_descricao(descricao):
    """
    Classifica uma descrição de serviço em um ou mais sistemas com base em palavras-chave.
    
    Args:
        descricao (str): Descrição do serviço.
    
    Returns:
        str: Sistemas encontrados na descrição separados por vírgulas ou 'Não Classificado'.
    """
    # Corrigir ortografia e converter para minúsculas
    descricao_corrigida = corrigir_ortografia(descricao.lower())
    
    # Dividir a descrição em palavras únicas para evitar redundâncias
    termos = set(descricao_corrigida.split())
    
    sistemas_encontrados = set()
    
    # Iterar sobre cada sistema e suas palavras-chave
    for sistema, palavras_chave in sistemas.items():
        # Utilizar interseção com fuzzy matching
        for palavra in palavras_chave:
            # Buscar termos com similaridade >= 80 usando process.extractOne para eficiência
            resultado_match = process.extractOne(palavra, termos, scorer=fuzz.ratio)
            if resultado_match and resultado_match[1] >= 80:
                sistemas_encontrados.add(sistema)
                break  # Passar para o próximo sistema após encontrar uma correspondência
    
    if not sistemas_encontrados:
        return 'Não Classificado'
    else:
        # Retornar os sistemas encontrados como uma string separada por vírgulas
        return ', '.join(sorted(sistemas_encontrados))

def classificar_falha_operacional(descricao):
    """
    Classifica se uma descrição de serviço é uma falha operacional ou não.
    
    Args:
        descricao (str): Descrição do serviço.
    
    Returns:
        str: 'Sim' se for falha operacional, 'Não' caso contrário.
    """
    # Corrigir ortografia e converter para minúsculas
    descricao_corrigida = corrigir_ortografia(descricao.lower())
    
    # Dividir a descrição em palavras únicas para evitar redundâncias
    termos = set(descricao_corrigida.split())
    
    # Iterar sobre cada categoria e suas palavras-chave
    for categoria, palavras_chave in falhas_operacionais.items():
        for palavra in palavras_chave:
            # Buscar termos com similaridade >= 80 usando process.extractOne para eficiência
            resultado_match = process.extractOne(palavra, termos, scorer=fuzz.ratio)
            if resultado_match and resultado_match[1] >= 80:
                return 'Sim'
    
    return 'Não'

# Leitura do arquivo Excel
# Substitua './builtin/base.xlsx' pelo caminho do seu arquivo
# E 'DESCRIÇÃO DO SERVIÇO' pelo nome exato da coluna que contém as descrições

# Exemplo:
caminho_entrada = './builtin/base.xlsx'
coluna_descricao = 'DESCRIÇÃO DO SERVIÇO'

try:
    df = pd.read_excel(caminho_entrada)
    print("Arquivo Excel lido com sucesso.")
except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_entrada}' não foi encontrado. Verifique o caminho e tente novamente.")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo Excel: {e}")

# Aplicação das classificações com barra de progresso

# Verificar se a coluna existe
if coluna_descricao not in df.columns:
    print(f"Erro: A coluna '{coluna_descricao}' não foi encontrada na planilha.")
else:
    # Inicializar a barra de progresso do tqdm
    tqdm.pandas(desc="Classificando Descrições")
    
    # Aplicar a função de classificação de sistemas
    df['Sistemas'] = df[coluna_descricao].progress_apply(classificar_descricao)
    
    # Aplicar a função de classificação de falha operacional
    df['Falha Operacional'] = df[coluna_descricao].progress_apply(classificar_falha_operacional)
    
    print("Classificação concluída.")

# Salvar o resultado em um novo arquivo Excel
# Substitua './builtin/sua_planilha_classificada.xlsx' pelo caminho desejado para salvar

caminho_saida = './builtin/sua_planilha_classificada.xlsx'

try:
    df.to_excel(caminho_saida, index=False)
    print(f"Classificação concluída e planilha salva como '{caminho_saida}'.")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo Excel: {e}")


StatementMeta(, 4d3739e1-fc1c-4da5-aa15-25a39f015caa, 5, Submitted, Running, Running)

Arquivo Excel lido com sucesso.


Classificando Descrições:   1%|▏         | 174/13465 [00:43<1:13:55,  3.00it/s]